In [1]:
import os
import sys 
import timeit
from scipy import special
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class NeuralNetwork:
    
    
    #初始化网络
    def __init__(self, inputnodes, hiddennodes, outputenodes, learningrate):
        
        #设置输入层、隐藏层、输出层节点
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputenodes
        
        #设置链接权重矩阵
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))  
                
        #设置学习率
        self.lr = learningrate
        
        #设置激活函数
        self.activation_function = lambda x: special.expit(x)

        pass
    
    
    #训练网络
    def train(self, inputs_list, targets_list):
        
        #把输入的列表转化为二维数组
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        #计算隐藏层
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #计算最终输出层信号
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        #计算误差
        output_errors = targets - final_outputs
        
        #计算隐含层节点反向传播的误差
        hidden_errors = np.dot(self.who.T, output_errors)
        
        #更新隐藏层和输出层之间的权重
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        #更新输入层和隐藏层之间的权重
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        
        pass
    
    
    #查询网络
    def query(self, inputs_list):
        
        #把输入的列表转化为二维数组
        inputs = np.array(inputs_list, ndmin=2).T
        
        #计算隐藏层
        hidden_inputs = np.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #计算最终输出层
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    

In [3]:
#设置输入层、隐藏层、输出层和学习率

input_nodes = 784
hidden_nodes = 200
output_nodes = 10

learning_rate = 0.1

In [4]:
#创建一个神经网络
n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [5]:
#读取训练数据集
training_data_file = open("mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [6]:
#训练数据集训练的次数
epochs = 5

#遍历训练数据集的所有数据
for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = np.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

In [7]:
#读取测试数据集
test_data_file = open("mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [8]:
#测试神经网络
scorecard = []
#遍历所有的测试数据集
for record in test_data_list:
        all_values = record.split(',')
        correct_label = int(all_values[0])
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        outputs = n.query(inputs)
        label = np.argmax(outputs)
        if(label == correct_label):
            scorecard.append(1)
        else:
            scorecard.append(0)
            pass
        pass

In [9]:
#评估性能
scorecard_array = np.asarray(scorecard)
print("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.973
